# Calculating Average Travel Time to Other Destination Categories
- Create a feature class for each destination type (healthcare, higher education, essential services) that contains the point locations, types, and capacity data.
- For each TAZ, find out how many (and ideally which) destinations of each type are within a buffer of the TAZ centroid.
- Flag each TAZ with whether it meets the threshold for the number of destinations within a buffer around the TAZ centroid for the type of destination.
- Use model skims for travel times from TAZ centroid to TAZ centroid for all of the following modes: Drive (SOV as proxy), Transit, Rapid Transit, Bus, Walk.
- Then pick the minimum time to reach a flagged TAZ and weight by TAZ population and use that for calculation of the average travel time for MPO.
- Then flag origin TAZs as whether their minimum distance to a flagged TAZ is within the average threshold for MPO.


Environment: base_py_37_omx_geop


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas
import openmatrix as omx

In [3]:
##Import all the CSVs that contain skims 
Walk_skim= omx.open_file('M:/LRTP/LRTP_AvgTT/out/Walk_skim.omx','r')
#Import AM
DAT_Boat_skim_AM = omx.open_file('M:/LRTP/LRTP_AvgTT/out/AM/A_DAT_for_Boat_tr_skim.omx','r')
DAT_CommRail_skim_AM = omx.open_file('M:/LRTP/LRTP_AvgTT/out/AM/A_DAT_for_CommRail_tr_skim.omx','r')
DAT_LocalBus_skim_AM = omx.open_file('M:/LRTP/LRTP_AvgTT/out/AM/A_DAT_for_LocalBus_tr_skim.omx','r')
DAT_RapidTransit_skim_AM = omx.open_file('M:/LRTP/LRTP_AvgTT/out/AM/A_DAT_for_RapidTransit_tr_skim2.omx','r')
WAT_Transit_skim_AM = omx.open_file('M:/LRTP/LRTP_AvgTT/out/AM\WAT_for_All_tr_skim.omx','r')
SOV_skim_AM = omx.open_file('M:/LRTP/LRTP_AvgTT/out/AM/SOV_skim.omx','r')

#put AM CSVs into Dictionary
skims_AM = {'DAT_Boat':DAT_Boat_skim_AM, 'DAT_CR':DAT_CommRail_skim_AM, 
            'DAT_LB':DAT_LocalBus_skim_AM, 'DAT_RT':DAT_RapidTransit_skim_AM, 
            'WAT_TR':WAT_Transit_skim_AM, 'SOV':SOV_skim_AM, 'Walk': Walk_skim}

#Import MD
DAT_Boat_skim_MD = omx.open_file('M:\LRTP\LRTP_AvgTT\out\MD\A_DAT_for_Boat_tr_skim.omx','r')
DAT_CommRail_skim_MD = omx.open_file('M:\LRTP\LRTP_AvgTT\out\MD\A_DAT_for_CommRail_tr_skim.omx','r')
DAT_LocalBus_skim_MD = omx.open_file('M:\LRTP\LRTP_AvgTT\out\MD\A_DAT_for_LocalBus_tr_skim.omx','r')
DAT_RapidTransit_skim_MD = omx.open_file('M:\LRTP\LRTP_AvgTT\out\MD\A_DAT_for_Rapid_Transit_tr_skim.omx','r')
WAT_Transit_skim_MD = omx.open_file('M:\LRTP\LRTP_AvgTT\out\MD\WAT_for_All_tr_skim.omx','r')
SOV_skim_MD = omx.open_file('M:\LRTP\LRTP_AvgTT\out\MD\SOV_skim.omx','r')

#put MD CSVs into Dictionary
skims_MD = {'DAT_Boat':DAT_Boat_skim_MD, 'DAT_CR':DAT_CommRail_skim_MD, 
            'DAT_LB':DAT_LocalBus_skim_MD, 'DAT_RT':DAT_RapidTransit_skim_MD, 
            'WAT_TR':WAT_Transit_skim_MD, 'SOV':SOV_skim_MD, 'Walk': Walk_skim}

In [16]:
##Import TAZ tables for Essential Services, Healthcare, and Higher Ed. 
dest_TAZs = pd.read_csv(r'M:/LRTP/LRTP_AvgTT/MPO_TAZ_ES_1m2.csv',header=0, sep=',',
                        usecols=['taz', 'ES_FLAG_ALL','HIED_Count5', 'HLTH_COUNT1m'])
#create table with column for each destination type that flags (0,1) which TAZs pass the destination threshold
dest_TAZs['HIED_FLAG']=[1 if x >=1 else 0 for x in dest_TAZs['HIED_Count5']]
dest_TAZs['HLTH_FLAG']=[1 if x >=1 else 0 for x in dest_TAZs['HLTH_COUNT1m']]

#bring in population data for later
tot_pop = pd.read_csv(r'M:/LRTP/LRTP_AvgTT/TAZ_Pop_CTPS.csv', header=0, sep=',', usecols=['TAZ_ID', 'Tot_Pop'])
dest_TAZs = dest_TAZs.merge(tot_pop, how='left', left_on='taz', right_on ='TAZ_ID')

dest_TAZs

,taz,ES_FLAG_ALL,HIED_Count5,HLTH_COUNT1m,HIED_FLAG,HLTH_FLAG,TAZ_ID,Tot_Pop
0,754,1,33,0,1,0,754,1070.0
1,1688,0,2,0,1,0,1688,2144.0
2,89,1,45,17,1,1,89,244.0
3,1385,1,2,0,1,0,1385,2945.0
4,727,1,39,2,1,1,727,1790.0
...,...,...,...,...,...,...,...,...
1896,2456,0,0,0,0,0,2456,1779.0
1897,2459,0,0,0,0,0,2459,2101.0
1898,2457,0,0,0,0,0,2457,2285.0
1899,2460,0,0,0,0,0,2460,1459.0


In [25]:
#make list of total TAZs (this is for row and column names for matrices)
tazs = dest_TAZs['taz'].tolist()
tazs.sort()
#make lists of only destination TAZs (this is for averaging)
ES_list = dest_TAZs[dest_TAZs['ES_FLAG_ALL']==1]['taz'].tolist()
HLTH_list = dest_TAZs[dest_TAZs['HLTH_FLAG']==1]['taz'].tolist()
HIED_list = dest_TAZs[dest_TAZs['HIED_FLAG']==1]['taz'].tolist()
len(HIED_list)

1594

In [6]:
##Sum the Skims to get Travel Time per Mode by Time of Day (loop through both AM and MD dictionaries)
AM_Tables = {}
MD_Tables = {}

for x in skims_AM.keys():
    if x != 'SOV' and x != 'Walk' and x != 'WAT_TR':
        AM_Tables[x] = pd.DataFrame((np.array(skims_AM[x]['Access Drive Time'])+np.array(skims_AM[x]['Access Walk Time'])
                        +np.array(skims_AM[x]['Dwelling Time'])+np.array(skims_AM[x]['Egress Drive Time'])
                        +np.array(skims_AM[x]['Egress Walk Time'])+np.array(skims_AM[x]['In-Vehicle Time'])
                        +np.array(skims_AM[x]['Initial Wait Time'])+np.array(skims_AM[x]['Transfer Penalty Time'])
                        +np.array(skims_AM[x]['Transfer Wait Time'])+np.array(skims_AM[x]['Transfer Walk Time']))[1:1902, 1:1902],
                                   index=tazs, columns=tazs).replace(-np.inf, np.nan)
    if x == 'WAT_TR':
        AM_Tables[x] = pd.DataFrame((np.array(skims_AM[x]['Access Walk Time'])
                        +np.array(skims_AM[x]['Dwelling Time'])
                        +np.array(skims_AM[x]['Egress Walk Time'])+np.array(skims_AM[x]['In-Vehicle Time'])
                        +np.array(skims_AM[x]['Initial Wait Time'])+np.array(skims_AM[x]['Transfer Penalty Time'])
                        +np.array(skims_AM[x]['Transfer Wait Time'])+np.array(skims_AM[x]['Transfer Walk Time']))[1:1902, 1:1902],
                                   index=tazs, columns=tazs).replace(-np.inf, np.nan)
    if x == 'SOV':
        AM_Tables[x] = pd.DataFrame(np.array(skims_AM[x]['CongTime'])[1:1902, 1:1902],
                                   index=tazs, columns=tazs).replace(-np.inf, np.nan)
    if x == 'Walk':
        AM_Tables[x] = pd.DataFrame((np.array(skims_AM[x]['WalkTime']))[1:1902, 1:1902],
                                   index=tazs, columns=tazs).replace(-np.inf, np.nan)
        
for x in skims_MD.keys():
    if x != 'SOV' and x != 'Walk' and x != 'WAT_TR':
        MD_Tables[x] = pd.DataFrame((np.array(skims_MD[x]['Access Drive Time'])+np.array(skims_MD[x]['Access Walk Time'])
                        +np.array(skims_MD[x]['Dwelling Time'])+np.array(skims_MD[x]['Egress Drive Time'])
                        +np.array(skims_MD[x]['Egress Walk Time'])+np.array(skims_MD[x]['In-Vehicle Time'])
                        +np.array(skims_MD[x]['Initial Wait Time'])+np.array(skims_MD[x]['Transfer Penalty Time'])
                        +np.array(skims_MD[x]['Transfer Wait Time'])+np.array(skims_MD[x]['Transfer Walk Time']))[1:1902, 1:1902],
                                   index=tazs, columns=tazs).replace(-np.inf, np.nan)
    if x == 'WAT_TR':
        MD_Tables[x] = pd.DataFrame((np.array(skims_MD[x]['Access Walk Time'])
                        +np.array(skims_MD[x]['Dwelling Time'])
                        +np.array(skims_MD[x]['Egress Walk Time'])+np.array(skims_MD[x]['In-Vehicle Time'])
                        +np.array(skims_MD[x]['Initial Wait Time'])+np.array(skims_MD[x]['Transfer Penalty Time'])
                        +np.array(skims_MD[x]['Transfer Wait Time'])+np.array(skims_MD[x]['Transfer Walk Time']))[1:1902, 1:1902],
                                   index=tazs, columns=tazs).replace(-np.inf, np.nan)
    if x == 'SOV':
        AM_Tables[x] = pd.DataFrame(np.array(skims_MD[x]['CongTime'])[1:1902, 1:1902],
                                   index=tazs, columns=tazs).replace(-np.inf, np.nan)
    if x == 'Walk':
        AM_Tables[x] = pd.DataFrame((np.array(skims_MD[x]['WalkTime']))[1:1902, 1:1902],
                                   index=tazs, columns=tazs).replace(-np.inf, np.nan)
    


C:\Users\matkinson\Anaconda3\envs\base_py_37_omx_geop\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: overflow encountered in add
  # This is added back by InteractiveShellApp.init_path()
C:\Users\matkinson\Anaconda3\envs\base_py_37_omx_geop\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: overflow encountered in add
C:\Users\matkinson\Anaconda3\envs\base_py_37_omx_geop\lib\site-packages\ipykernel_launcher.py:33: RuntimeWarning: overflow encountered in add
C:\Users\matkinson\Anaconda3\envs\base_py_37_omx_geop\lib\site-packages\ipykernel_launcher.py:40: RuntimeWarning: overflow encountered in add


In [7]:
#close everything! 
Walk_skim.close()

DAT_Boat_skim_AM.close()
DAT_CommRail_skim_AM.close()
DAT_LocalBus_skim_AM.close()
DAT_RapidTransit_skim_AM.close()
WAT_Transit_skim_AM.close()
SOV_skim_AM.close()

DAT_Boat_skim_MD.close()
DAT_CommRail_skim_MD.close()
DAT_LocalBus_skim_MD.close()
DAT_RapidTransit_skim_MD.close()
WAT_Transit_skim_MD.close()
SOV_skim_MD.close()

In [9]:
MD_Tables['WAT_TR']

,1,2,3,4,5,6,7,8,9,10,...,2636,2637,2638,2639,2640,2641,2642,2643,2644,2645
1,NaN,15.890332,15.067829,16.681107,13.844900,22.188412,20.573730,18.769184,15.226395,16.328690,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,143.467194
2,13.853310,NaN,11.143913,12.757190,13.426848,18.264496,16.264828,18.039385,13.737673,14.839968,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,139.941437
3,13.653575,12.071397,NaN,10.401732,11.013784,15.909040,13.909370,15.683928,14.437603,17.006336,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,141.813904
4,17.963009,16.380831,13.097890,NaN,15.323218,20.218473,18.218803,19.993361,18.747036,18.363400,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,145.211273
5,14.888954,13.306777,10.023836,11.637113,NaN,17.144421,15.144750,16.919308,15.672982,19.136524,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,140.981781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2641,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2642,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2643,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2644,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#factor WAT with DAT (weight by # of trips?)

#Create new table in dictionary: All Transit
    #take the average - weight it by # of trips for each type of transit
    #take the lowest?

In [52]:
##Calculate Averages
    #weighted average of all the lowest travel times from each TAZ to closest (timewise) destination TAZ 
        #(ones that pass the destination threshold (flagged)), weighted by population of the origin TAZ.
    #do this for all modes and TOD - should end up with a number each for SOV, All Transit, Rapid Transit, Local Bus, Commuter Rail, Boat, and Walking for AM and MD.

    #want to end up with a table for each mode for each dest type with these columns: Origin TAZ, Dest TAZ, Time, Tot_Pop
    #need to do AM for Healthcare and Essential Services, do MD for HiEd. (three tables per mode)
ES_RT = AM_Tables['DAT_RT'].loc[:,ES_list] #restrict columns to just what is an ES destination TAZ
ES_RT['Min_Dest_TAZ'] = ES_RT.idxmin(axis=1) #new column with the column name of the smallest time in each row
ES_RT['TravelTime'] = ES_RT.min(1) #new column with the min val in each row (corresponds with dest TAZ above)
ES_RT = ES_RT.reset_index() #turn index into origin taz field
ES_RT = ES_RT.rename(columns = {'index':'origin taz'})

ES_RT = ES_RT[['origin taz', 'Min_Dest_TAZ', 'TravelTime']] #restrict to a manageable table

#if want to not restrict to just transit (e.g. if O and D are same TAZ)
ES_RT.loc[ES_RT['origin taz'].isin(ES_list), 'Min_Dest_TAZ'] = ES_RT['origin taz'] #if taz is already a dest taz, set D taz to O taz
ES_RT.loc[ES_RT['origin taz'] == ES_RT['Min_Dest_TAZ'], 'TravelTime'] = None #if the taz is a dest taz, set TT to null
#note here: so there will never be a transit value if the dest taz is the same as the origin taz
#will need to replace with Walk or SOV here - depends on DAT or WAT?
ES_RT

,origin taz,Min_Dest_TAZ,TravelTime
0,1,1,NaN
1,2,2,NaN
2,3,3,NaN
3,4,4,NaN
4,5,5,NaN
...,...,...,...
1896,2641,2641,NaN
1897,2642,2642,NaN
1898,2643,1089,49.1273
1899,2644,2644,NaN


In [ ]:
##Make Flag Tables
#if the TAZ has access to a destination TAZ within the MPO average for that mode, flag.
    #How do I make this into a table?  - maybe copy the TAZ to TAZ table and clear its contents, then just go down the column of all the destination TAZs and query the skim table for that mode and TOD.
    #where column name is in list (aka field in the table that flags destination TAZs for each type)



In [53]:
AM_Tables['DAT_RT']

,1,2,3,4,5,6,7,8,9,10,...,2636,2637,2638,2639,2640,2641,2642,2643,2644,2645
1,NaN,18.112869,18.558861,21.636301,21.242023,27.143608,25.713810,26.166307,22.623516,23.725811,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,18.587736,NaN,17.545006,20.622444,19.928167,26.129751,23.979954,24.432451,20.889660,21.991955,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,17.837376,16.348654,NaN,19.872084,19.177807,25.379391,23.229595,23.682091,20.139299,21.241596,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,18.313158,16.824436,17.270428,NaN,19.653589,25.855173,23.705376,24.157873,20.615082,21.717377,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,18.681587,17.192865,17.638855,20.716295,NaN,26.223602,24.073807,24.526302,20.983511,22.085808,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2641,175.986252,174.497543,174.943527,178.020966,173.876694,183.528275,178.348465,178.800980,175.258179,176.360474,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2642,173.305374,171.816635,172.262634,175.340073,171.195786,180.847382,175.667587,176.120071,172.577286,173.679581,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2643,171.581711,170.092972,170.538971,173.616409,169.472122,179.123718,173.943924,174.396408,170.853622,171.955917,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2644,170.194839,168.706131,169.152115,172.229553,168.085281,177.736862,172.557068,173.009567,169.466782,170.569077,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
